In [16]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import cv2
import pandas as pd
import math
import pandas as pd
from skimage import morphology, measure
import pickle

In [17]:
i = 20 # image number
frame = 100 # push the boundary of roi inwards

In [18]:
def loadImageAsArray(path):
    return np.array(Image.open(path))

In [19]:
paths = [os.path.join('train',file) for file in os.listdir('train')]

In [20]:
# organize label
file_path = 'JustRAIGS_Train_labels.csv'
alldata = pd.read_csv(file_path, usecols = range(27))
positive_filenames = alldata[alldata['Final Label'] == 'RG']['Eye ID'].tolist()

In [21]:

def convertGray(i):
    name = positive_filenames[i] + '.JPG' # 1, 
    arr = loadImageAsArray('train/'+ name)
    gray_image = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    return gray_image

In [22]:
def remove_small_areas(image, min_size):
    # Label connected regions
    labels = measure.label(image, connectivity=2) 

    # Calculate region properties
    props = measure.regionprops(labels)

    # Create a mask for small regions
    mask = np.zeros_like(image, dtype=bool)
    for prop in props:
        if prop.area < min_size:
            mask[labels == prop.label] = True

    # Remove small regions
    result = image.copy()
    result[mask] = 0

    return result

In [23]:
def find_y(center_x, center_y, radius, line_x):
    y1 = math.sqrt(radius**2 - (line_x-center_x)**2) + center_y
    y2 = -1* math.sqrt(radius**2 - (line_x-center_x)**2) + center_y
    return y1, y2

In [24]:
def find_x(center_x, center_y, radius, line_y):
    x1 = math.sqrt(radius**2 - (line_y-center_y)**2) + center_x
    x2 = -1* math.sqrt(radius**2 - (line_y-center_y)**2) + center_x
    return x1, x2

In [25]:
def find_center(roi, max_loc):
    roi = cv2.medianBlur(roi, 35)
    x_right = 0
    x_left = 0
    y_up = 0
    y_down = 0
    x = max_loc[1]
    y = max_loc[0]
    i = 0
    while roi[y, x+i] > 0:
        x_right = x+i
        i = i+1
    i = 0

    while roi[y, x-i] > 0:
        x_left = x-i
        i = i + 1
    i = 0
    while roi[y+i, x] > 0:
        y_down = y+1
        i = i+1
    i = 0
    while roi[y-i, x] > 0:
        y_up = y-i
        i = i + 1
    center = [(y_up + y_down)/2, (x_left + x_right)/2]
    y_radius = abs(y_up-y_down)
    x_radius = abs(x_left - x_right)
    radius = max(x_radius, y_radius)
    return center, radius

For large cup (disc to cup ratio), locate cup and disc

In [26]:
def getROI(roi, frame, gray_image):
    y_all, x_all = gray_image.shape
    # Find ROI
    gray_image_new = gray_image[round(y_all*0.2):round(y_all*0.66), 0:x_all]
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray_image_new)
    print(min_val, max_val, min_loc, max_loc)
    y_all_new, x_all_new = gray_image_new.shape
    roi_size_x = x_all_new//3 # for cupping, only the disc
    roi_size_y = y_all_new//2
    x_start = max_loc[0] - roi_size_x//2 
    y_start = max_loc[1] - roi_size_y//2
    x_end = max_loc[0] + roi_size_x//2
    y_end = max_loc[1] + roi_size_y//2
    roi = gray_image_new[y_start:y_end, x_start:x_end]
    return  x_start, y_start

In [30]:
def findCFR(roi, frame, gray_image):
    y_all, x_all = roi.shape
    # get min enclosing circle for disc
    x_start, y_start= getROI(gray_image)
    roi_disc = roi[frame: y_all - frame, frame: x_all-frame]
    _, max_val, _, max_loc = cv2.minMaxLoc(roi_disc)

    plt.imshow(roi_disc)
    plt.scatter(*max_loc,c='r')
    plt.show()
    roi_disc[ roi_disc < (max_val - 60)] = 0
    plt.imshow(roi_disc)
    _, max_val, _, max_loc = cv2.minMaxLoc(roi_disc)
    canny_disc = cv2.Canny( roi_disc, 0, 200)
    points_disc = np.argwhere(canny_disc>0)
    print(points_disc)
    center_disc, radius_disc = cv2.minEnclosingCircle(points_disc)
    print('center:', center_disc, 'radius:', radius_disc)

    result_disc= roi.copy()
    x_disc = int(center_disc[1]+ x_start) 
    y_disc = int(center_disc[0]+ y_start)
    rad_disc = int(radius_disc)
    cv2.circle(result_disc, [y_disc,x_disc], rad_disc, (255,255,255), 1)
    plt.imshow(result_disc)

    roi_cup = roi[frame: y_all - frame, frame: x_all-frame]
    _, max_val,_, max_loc = cv2.minMaxLoc(roi_cup)
    plt.scatter(*max_loc,c='r')
    plt.show()

    roi_cup [roi_cup  < (max_val - 20)] = 0
    canny_cup = cv2.Canny(roi_cup , 0, 200)
    points_cup = np.argwhere(canny_cup>0)

    # get min enclosing circle for cup
    center_cup, radius_cup = cv2.minEnclosingCircle(points_cup)
    print('center:', center_cup, 'radius:', radius_cup)
    result_cup = roi.copy()
    x = int(center_disc[1])
    y = int(center_disc[0])
    rad = int(radius_cup)
    cv2.circle(result_cup, [y,x], rad, (255,255,255), 1)
    CDR = radius_cup/radius_disc
    print(center_cup, center_disc, radius_cup, radius_disc)
    # calculate I, S, N, T (Normal neuroretinal rim: The neuroretinal rim is the most important parameter of the optic disk evaluation. 
    # The optic disk is vertically oval and the cup is horizontally oval thus the rim has a characteristic configuration where the inferior (I) 
    # rim is the widest, followed by the superior (S) and nasal rims (N) and the temporal (T) rim is the thinnest. 
    # This is the ‘ISNT rule' which helps to determine glaucomatous changes in the disk glaucoma. On an average, 
    # the inferior rim is 18% thicker than the superior rim

    line_x = center_cup[1]
    center_x_disc = center_disc[1]
    center_y_disc = center_disc[0]
    center_x_cup = center_cup[1]
    center_y_cup = center_cup[0]
    y_disc_s, y_disc_i = find_y(center_x_disc, center_y_disc, radius_disc, line_x)
    y_cup_s, y_cup_i = find_y(center_x_cup, center_y_cup, radius_cup, line_x)
    S =  abs(y_cup_s - y_disc_s)
    I =  abs(y_disc_i - y_cup_i)
    line_y = center_cup[0]
    center_x_disc = center_disc[1]
    center_y_disc = center_disc[0]
    center_x_cup = center_cup[1]
    center_y_cup = center_cup[0]
    x_disc_l, x_disc_r = find_x(center_x_disc, center_y_disc, radius_disc, line_y)
    x_cup_l, x_cup_r = find_x(center_x_cup, center_y_cup, radius_cup, line_y)
    T = abs(x_cup_l - x_disc_l)
    N = abs(x_disc_r - x_cup_r)
    return  CDR, I, S, N, T

In [31]:
def loadGray(i):
    name = positive_filenames[i] + '.npy' # 1, 
    arr = np.load('preprocessed/'+ name)
    gray_image = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    return gray_image

In [32]:
d = {}
for i in range(3):#(len(positive_filenames)):
    try:
        roi = loadGray(i)
        gray_image = convertGray(i)
        findCFR(roi, frame, gray_image)
        CDR, S, I, T, N = findCFR(roi, frame, gray_image)
        d[positive_filenames[i]] = [CDR,S,I,T,N]
    except:
        d[positive_filenames[i]] = [0,0,0,0,0]
# pickle.dump(d,open('cup.pkl','wb'))